In [7]:
import json,requests
marquez_url = "http://api:5000" ## this may depend on your local setup
if (requests.get("{}/api/v1/namespaces".format(marquez_url)).status_code == 200):
    print("Marquez is OK.")
else:
    print("Cannot connect to Marquez")

Marquez is OK.


Marquez URL is http://localhost:3000

In [8]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder.master('local')
         .appName('Jelly Data Mesh')
         .config('spark.extraListeners', 'io.openlineage.spark.agent.OpenLineageSparkListener')
         .config('spark.jars.packages', 'io.openlineage:openlineage-spark:0.20.6')
         .config('spark.openlineage.transport.type', 'http')
         .config('spark.openlineage.transport.url', 'http://api:5000')    
         .config('spark.openlineage.namespace', 'jelly-world')
         .getOrCreate())

spark.sparkContext.setLogLevel('DEBUG')

## Initialise some datasets

In [23]:
spark.sql('DROP TABLE IF EXISTS jelly_colors')
spark.sql('DROP TABLE IF EXISTS jelly_transactions')
spark.sql('DROP TABLE IF EXISTS jelly_users')

DataFrame[]

In [28]:
!rm -rf /home/jovyan/notebooks/spark-warehouse/jelly_*

In [29]:
import uuid
greenUuid = str(uuid.uuid4())
pawelUuid = str(uuid.uuid4())
maciejUuid = str(uuid.uuid4());

spark.createDataFrame([
    {'color_uuid': greenUuid, 'name': 'green'},
    {'color_uuid': str(uuid.uuid4()), 'name': 'yellow'},
]).write.mode("overwrite").saveAsTable('jelly_colors')

spark.createDataFrame([
    {'user_uuid': pawelUuid, 'name': 'pawel@jellyworld.pl'},
    {'user_uuid': maciejUuid, 'name': 'maciej@jellyworld.pl'},
]).write.mode("overwrite").saveAsTable('jelly_users')

spark.createDataFrame([
    {'user_uuid': pawelUuid, 'color_uuid': greenUuid }
]).write.mode("overwrite").saveAsTable('jelly_favorite_colors')

spark.createDataFrame([
    {
        'from_user_uuid': pawelUuid, 
        'to_user_uuid': maciejUuid, 
        'exchanched_color_uuid': greenUuid, 
        'amount_exchanged': 5, 
        'lat': 52.2278015 , 
        'lng': 21.0022384, 
        'exchange_date': '2023-02-29 12:00:00.000' 
    }
]).write.mode("overwrite").saveAsTable('jelly_transactions')

## Active daily users report table

In [36]:
spark.sql('''
  SELECT u.user_uuid, u.name, SUM(amount_exchanged)
  FROM jelly_users u
  JOIN jelly_transactions t ON (u.user_uuid=t.from_user_uuid OR u.user_uuid=t.to_user_uuid) 
  GROUP BY u.user_uuid, u.name
''').write.mode("overwrite").saveAsTable('jelly_daily_active_users')

## Active daily users that like green jellys

In [39]:
spark.sql('''
  SELECT u.name, jelly_colors.name as color
  FROM jelly_daily_active_users u
  JOIN jelly_favorite_colors ON jelly_favorite_colors.user_uuid = u.user_uuid
  JOIN jelly_colors ON jelly_colors.color_uuid= jelly_favorite_colors.color_uuid
  WHERE jelly_colors.name = 'green'
''').write.mode("overwrite").saveAsTable('jelly_daily_green')

Marquez:
 * Job in [Marquez UI](http://localhost:3000/lineage/job/jelly-world/jelly_data_mesh.execute_create_data_source_table_as_select_command).
 * [Lineage in Marquez API](http://localhost:3000/api/v1/column-lineage?nodeId=dataset:file:%2Fhome%2Fjovyan%2Fnotebooks%2Fspark-warehouse%2Fjelly_daily_green)